# 推理示例

In [1]:
from mindformers import AutoTokenizer, AutoModel


tokenizer = AutoTokenizer.from_pretrained("glm2_6b")
model = AutoModel.from_pretrained("glm2_6b")

query = "你好"

prompted_inputs = tokenizer.build_prompt(query)
input_tokens = tokenizer([prompted_inputs])

outputs = model.generate(input_tokens["input_ids"], max_length=100)
response = tokenizer.decode(outputs)[0]
print(response)

2023-08-19 12:28:47,192 - mindformers - INFO - Config in the yaml file ./checkpoint_download/glm2/glm2_6b.yaml are used for tokenizer building.
2023-08-19 12:28:47,232 - mindformers - INFO - Load the tokenizer name ChatGLM2Tokenizer from the ./checkpoint_download/glm2/glm2_6b.yaml
2023-08-19 12:28:47,255 - mindformers - INFO - config in the yaml file ./checkpoint_download/glm2/glm2_6b.yaml are used for tokenizer building.
2023-08-19 12:28:47,277 - mindformers - WARNING - Can't find the tokenizer_config.json in the file_dict. The content of file_dict is : {}
2023-08-19 12:28:47,287 - mindformers - INFO - build tokenizer class name is: ChatGLM2Tokenizer using args {'bos_token': '<sop>', 'eos_token': '<eop>', 'end_token': '</s>', 'mask_token': '[MASK]', 'gmask_token': '[gMASK]', 'pad_token': '<pad>', 'unk_token': '<unk>', 'vocab_file': './checkpoint_download/glm2/tokenizer.model'}.
2023-08-19 12:28:47,367 - mindformers - INFO - ChatGLM2Tokenizer Tokenizer built successfully!
2023-08-19 12

2023-08-19 12:31:41,733 - mindformers - INFO - weights in ./checkpoint_download/glm2/glm2_6b.ckpt are loaded
2023-08-19 12:31:41,735 - mindformers - INFO - model built successfully!
[Round 1]

问：你好

答： 你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。


# 推理部署示例 - 对话机器人
建议在终端运行脚本chatglm2_demo.py

In [9]:
import os
import platform
import signal
import readline

import time
import mindspore as ms
from mindformers import AutoConfig, AutoModel, AutoTokenizer


ms.set_context(mode=ms.GRAPH_MODE, device_target="Ascend", device_id=0)

config = AutoConfig.from_pretrained("glm2_6b")
# 可以在此使用下行代码指定自定义权重进行推理，默认使用自动从obs上下载的预训练权重
# config.checkpoint_name_or_path = "/path/to/glm2_6b_finetune.ckpt"
config.use_past = True
model = AutoModel.from_config(config)
tokenizer = AutoTokenizer.from_pretrained("glm2_6b")

os_name = platform.system()
clear_command = 'cls' if os_name == 'Windows' else 'clear'
stop_stream = False


def build_prompt(history):
    prompt = "欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序"
    for query, response in history:
        prompt += f"\n\n用户：{query}"
        prompt += f"\n\nChatGLM2-6B：{response}"
    return prompt


def signal_handler(signal, frame):
    global stop_stream
    stop_stream = True


def main():
    history = []
    global stop_stream
    print("欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序")
    while True:
        query = input("\n用户：")
        if query.strip() == "stop":
            break
        if query.strip() == "clear":
            history = []
            os.system(clear_command)
            print("欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序")
            continue
        count = 0
        
        # prompt: [Round 1]\n\n问：{query}\n\n答： 
        prompted_inputs = tokenizer.build_prompt(query)
        inputs = tokenizer(prompted_inputs)['input_ids']
        outputs = model.generate(inputs, max_length=128)
        response = tokenizer.decode(outputs)[0].split("答： ")[-1]
        history = history + [(query, response)]

        if stop_stream:
            stop_stream = False
            break
        else:
            count += 1
            if count % 8 == 0:
                os.system(clear_command)
                print(build_prompt(history), flush=True)
                signal.signal(signal.SIGINT, signal_handler)
        os.system(clear_command)
        print(build_prompt(history), flush=True)


if __name__ == "__main__":
    main()

2023-11-12 03:33:32,005 - mindformers[mindformers/models/base_model.py:110] - INFO - start to read the ckpt file: 12487185277


2023-11-12 03:35:09,629 - mindformers[mindformers/models/base_model.py:116] - INFO - weights in ./checkpoint_download/glm2/glm2_6b.ckpt are loaded
2023-11-12 03:35:09,644 - mindformers[mindformers/auto_class.py:291] - INFO - model built successfully!
2023-11-12 03:35:15,251 - mindformers[mindformers/auto_class.py:702] - INFO - Config in the yaml file ./checkpoint_download/glm2/glm2_6b.yaml are used for tokenizer building.
2023-11-12 03:35:16,582 - mindformers[mindformers/auto_class.py:709] - INFO - Load the tokenizer name ChatGLM2Tokenizer from the ./checkpoint_download/glm2/glm2_6b.yaml
2023-11-12 03:35:16,633 - mindformers[mindformers/models/base_tokenizer.py:1979] - INFO - config in the yaml file ./checkpoint_download/glm2/glm2_6b.yaml are used for tokenizer building.
2023-11-12 03:35:16,682 - mindformers[mindformers/models/base_tokenizer.py:1989] - WARNING - Can't find the tokenizer_config.json in the file_dict. The content of file_dict is : {}
2023-11-12 03:35:16,684 - mindformers


用户： 你好


2023-11-12 03:40:47,389 - mindformers[mindformers/generation/text_generator.py:442] - INFO - total time: 292.39666175842285 s; generated tokens: 28 tokens; generate speed: 0.09576032719256387 tokens/s
欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序

用户：你好

ChatGLM2-6B：你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。



用户： 请介绍下华为


2023-11-12 03:43:40,087 - mindformers[mindformers/generation/text_generator.py:442] - INFO - total time: 5.809668779373169 s; generated tokens: 108 tokens; generate speed: 18.589700050275948 tokens/s
欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序

用户：你好

ChatGLM2-6B：你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

用户：请介绍下华为

ChatGLM2-6B：华为是一家总部位于中国的全球知名科技公司,成立于1987年,是全球领先的信息与通信技术(ICT)解决方案供应商之一。

华为的业务范围涵盖了网络、终端、云计算、软件、芯片等多个领域,旗下的智能手机、电脑、平板电脑等消费电子产品在国内外市场上都享有较高的声誉。此外,华为还在5G、人工智能、云计算等领域进行了大量的投资和研发,致力于成为全球领先的数字化技术领导者。

华为以其高品质的产品和服务赢得了全球客户的信任和好评,也曾因其领先



用户： 请介绍下chatglm2


2023-11-12 03:44:03,998 - mindformers[mindformers/generation/text_generator.py:442] - INFO - total time: 2.8868305683135986 s; generated tokens: 54 tokens; generate speed: 18.70563537490363 tokens/s
欢迎使用 ChatGLM2-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序

用户：你好

ChatGLM2-6B：你好👋！我是人工智能助手 ChatGLM2-6B，很高兴见到你，欢迎问我任何问题。

用户：请介绍下华为

ChatGLM2-6B：华为是一家总部位于中国的全球知名科技公司,成立于1987年,是全球领先的信息与通信技术(ICT)解决方案供应商之一。

华为的业务范围涵盖了网络、终端、云计算、软件、芯片等多个领域,旗下的智能手机、电脑、平板电脑等消费电子产品在国内外市场上都享有较高的声誉。此外,华为还在5G、人工智能、云计算等领域进行了大量的投资和研发,致力于成为全球领先的数字化技术领导者。

华为以其高品质的产品和服务赢得了全球客户的信任和好评,也曾因其领先

用户：请介绍下chatglm2

ChatGLM2-6B：ChatGLM2-6B 是一个人工智能助手，基于清华大学 KEG 实验室与智谱 AI 于 2023 年联合训练的语言模型 GLM2-6B 开发而成，可以针对用户的问题和要求提供适当的答复和支持。



用户： stop
